In [ ]:
!pip install pydot graphviz

In [ ]:
# for loading/processing the images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle
from kaggle_datasets import KaggleDatasets

!pip install pydot graphviz

# Loading Monet images

In [ ]:
path = "../input/gan-getting-started/monet_jpg/"

# this list holds all the image filename
monet_images = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.jpg'):
          # adds only the image files to the monet_images list
            monet_images.append(path + file.name)

In [ ]:
monet_images[0]

In [ ]:
# load the image as a 224x224 array
img = load_img(monet_images[0], target_size=(224,224))
# convert from 'PIL.Image.Image' to numpy array
img = np.array(img)

print(img.shape)

In [ ]:
reshaped_img = img.reshape(1,224, 224, 3)
print(reshaped_img.shape)

In [ ]:
x = preprocess_input(reshaped_img)

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import time
from PIL import Image
%matplotlib inline

# Extracting features using trained model

In [ ]:
# load model
model = VGG16()
# remove the output layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

In [ ]:
features = model.predict(x)
print(features.shape)

In [ ]:
# load the model first and pass as an argument
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [ ]:
monet_images[0]

In [ ]:
data = {}
p = "monet_images.pkl"

# lop through each image in the dataset
for monet_image in monet_images:
    # try to extract the features and update the dictionary
    feat = extract_features(monet_image,model)
    data[monet_image] = feat
          
 
# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))
feat.shape
(210, 1, 4096)

# reshape so that there are 210 samples of 4096 vectors
feat = feat.reshape(-1,4096)
feat.shape
(210, 4096)

unique_labels = list(set(list(range(30))))

# Using PCA to lower dimentions

In [ ]:
pca = PCA(n_components=100, random_state=22)
pca.fit(feat)
x = pca.transform(feat)

In [ ]:
kmeans = KMeans(n_clusters=len(unique_labels),n_jobs=-1, random_state=22)
kmeans.fit(x)

In [ ]:
# holds the cluster id and the images { id: [images] }
groups = {}
for file, cluster in zip(filenames,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

In [ ]:
groups[0]

In [ ]:
import cv2

imgs = groups[0]

plt.figure()

#subplot(r,c) provide the no. of rows and columns
f, axarr = plt.subplots(len(imgs),1, figsize = (len(imgs)*30,30))

for i, img_path in enumerate(imgs):
    axarr[i].imshow(cv2.imread(imgs[i]))

# Creating 30 Monet images dataset

In [ ]:
# we choose image 0 in every cluster as a representitive
monet_dataset_paths = []
for i in range(len(groups)):
    monet_dataset_paths.append(groups[i][0])
print(monet_dataset_paths)
len(monet_dataset_paths)

In [ ]:
import os
import time
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, Flatten, ReLU, BatchNormalization, Conv2DTranspose, Dense
import albumentations as A
import cv2
import glob
from functools import partial

from IPython.display import clear_output
from kaggle_datasets import KaggleDatasets
# import matplotlib.animation as animation

AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    INPUT_PATH = KaggleDatasets().get_gcs_path()
else:
    strategy = tf.distribute.get_strategy()
    INPUT_PATH = "../input/gan-getting-started"

print("REPLICAS: ", strategy.num_replicas_in_sync)
print("Input Path:", INPUT_PATH)

In [ ]:
photo_tfrec_files = tf.io.gfile.glob(INPUT_PATH+"/photo_tfrec/*.tfrec")

In [ ]:
feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
}

In [ ]:
def parse_tfrecord(record):
    features = tf.io.parse_single_example(record, feature_description)
    
    image = features['image']
    image = tf.io.decode_image(image)
    image = tf.reshape(image, (256, 256, 3))
    
    return image

In [ ]:
def parse_img(image_name):
    image = cv2.imread(image_name)
    return image

In [ ]:
monet_dataset_paths

In [ ]:
test_img = parse_img(monet_dataset_paths[0])
print(type(test_img))

In [ ]:
p_transformation = 0.1
transforms = A.Compose(
    [
#         A.Blur(p=p_transformation, blur_limit=(5, 5)),
#         A.CLAHE(p=p_transformation, clip_limit=(10, 10), tile_grid_size=(3, 3)),
#         #A.CenterCrop(p=p_transformation, height=100, width=150),
#         A.ChannelDropout(p=p_transformation, channel_drop_range=(1, 2), fill_value=0),
#         A.ChannelShuffle(p=p_transformation),
        A.RandomCrop(p=p_transformation, height=150, width=150),
        A.Cutout(p=p_transformation, num_holes=8, max_h_size=15, max_w_size=15),
#         A.Downscale(p=p_transformation, scale_min=0.01, scale_max=0.20, interpolation=0),
#         A.Equalize(p=p_transformation, mode='cv', by_channels=True),
        A.HorizontalFlip(p=p_transformation),
        A.VerticalFlip(p=p_transformation),
        A.Flip(p=p_transformation),
#         A.GaussNoise(p=p_transformation, var_limit=(500.0, 500.0)),
#         A.GridDistortion( p=p_transformation, num_steps=15, distort_limit=(-2., 2.), interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None),
#         A.HueSaturationValue(p=p_transformation, 
#             hue_shift_limit=(-100, 100), 
#             sat_shift_limit=(-100, 100), 
#             val_shift_limit=(-100, 100)),
#         A.ISONoise(p=p_transformation, intensity=(0.0, 2.0), color_shift=(0.0, 1.0)),
#         A.ImageCompression(p=p_transformation, quality_lower=0, quality_upper=10, compression_type=0),
#         A.InvertImg(p=p_transformation),
#         A.JpegCompression(p=p_transformation, quality_lower=0, quality_upper=10),
#         A.MotionBlur(p=p_transformation, blur_limit=(3, 50)),
#         A.MultiplicativeNoise(p=p_transformation, multiplier=(0.1, 5.0), per_channel=True, elementwise=False),
    ]
)

In [ ]:
def aug_fn(image, img_size):
    data = {"image":image}
    aug_data = transforms(**data)
    aug_img = aug_data["image"]
    aug_img = tf.cast(aug_img, tf.float32)
    aug_img = tf.image.resize(aug_img, size=[img_size, img_size], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return aug_img

def process_data(image, img_size):
    aug_img = tf.numpy_function(func=aug_fn, inp=[image, img_size], Tout=tf.float32)
    return aug_img

def random_jitter(image):
    image = tf.image.resize(image, [286, 286], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    image = tf.image.random_crop(image, size=[256,256, 3])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_saturation(image, 0.7, 1.2)
    return image

BATCH_SIZE = 30
BUFFER_SIZE = 1000

print("Batch Size:", BATCH_SIZE)
print("Buffer Size:", BUFFER_SIZE)

def normalize(image):
    image = tf.image.resize(image, [256,256], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1.0
    
    return image

def normalize_monet(image):
    image = (image / 127.5) - 1.0
    return image

monet_dataset = list(map(parse_img, monet_dataset_paths))
monet_dataset = tf.data.Dataset.from_tensor_slices(monet_dataset)
monet_dataset = monet_dataset.map(partial(process_data, img_size=256), num_parallel_calls=AUTOTUNE)
monet_dataset = monet_dataset.map(normalize_monet, num_parallel_calls=AUTOTUNE)
monet_dataset = monet_dataset.shuffle(BUFFER_SIZE)
monet_dataset = monet_dataset.batch(BATCH_SIZE)
monet_dataset = monet_dataset.prefetch(AUTOTUNE)

# monet_dataset = map(parse_img, monet_jpg_files)
# monet_dataset_list = list(map(random_jitter_monet, monet_dataset))

# monet_dataset = tf.data.Dataset.from_tensor_slices(monet_dataset_list)
# monet_dataset = monet_dataset.map(normalize, num_parallel_calls=AUTOTUNE)
# monet_dataset = monet_dataset.repeat()
# monet_dataset = monet_dataset.shuffle(BUFFER_SIZE)
# monet_dataset = monet_dataset.batch(BATCH_SIZE)
# monet_dataset = monet_dataset.prefetch(AUTOTUNE)

In [ ]:
photo_dataset = tf.data.TFRecordDataset(photo_tfrec_files)
photo_dataset = photo_dataset.map(parse_tfrecord, num_parallel_calls=AUTOTUNE)
# photo_dataset = photo_dataset.cache("/kaggle/tmp/photo")
photo_dataset = photo_dataset.map(random_jitter, num_parallel_calls=AUTOTUNE)
photo_dataset = photo_dataset.map(normalize, num_parallel_calls=AUTOTUNE)
photo_dataset = photo_dataset.shuffle(BUFFER_SIZE)
photo_dataset = photo_dataset.batch(BATCH_SIZE)
photo_dataset = photo_dataset.prefetch(AUTOTUNE)

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    
    result = tf.keras.Sequential()
    result.add(tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',kernel_initializer=initializer, use_bias=False))
    
    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())

    result.add(tf.keras.layers.LeakyReLU())
    return result

In [ ]:
def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    
    result = tf.keras.Sequential()
    result.add(
        tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))
    
    result.add(tf.keras.layers.BatchNormalization())
    
    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))
        
    result.add(tf.keras.layers.ReLU())
    
    return result

In [ ]:
def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)
    
    inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
    
    down1 = downsample(64, 4, False)(inp) # (bs, 128, 128, 64)
    down2 = downsample(128, 4)(down1) # (bs, 64, 64, 128)
    down3 = downsample(256, 4)(down2) # (bs, 32, 32, 256)
    
    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
    conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                  kernel_initializer=initializer,
                                  use_bias=False)(zero_pad1) # (bs, 31, 31, 512)
    
    batchnorm1 = tf.keras.layers.BatchNormalization()(conv)
    
    leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)
    
    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)
    
    last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                  kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)
    
    return tf.keras.Model(inputs=[inp], outputs=last)

In [ ]:
def Generator():
    inputs = tf.keras.layers.Input(shape=[256,256,3])
    
    down_stack = [
        downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
        downsample(128, 4), # (bs, 64, 64, 128)
        downsample(256, 4), # (bs, 32, 32, 256)
        downsample(512, 4), # (bs, 16, 16, 512)
        downsample(512, 4), # (bs, 8, 8, 512)
        downsample(512, 4), # (bs, 4, 4, 512)
        downsample(512, 4), # (bs, 2, 2, 512)
        downsample(512, 4), # (bs, 1, 1, 512)
    ]
    
    up_stack = [
        upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
        upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
        upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
        upsample(512, 4), # (bs, 16, 16, 1024)
        upsample(256, 4), # (bs, 32, 32, 512)
        upsample(128, 4), # (bs, 64, 64, 256)
        upsample(64, 4), # (bs, 128, 128, 128)
    ]
    
    initializer = tf.random_normal_initializer(0., 0.02)
    last = tf.keras.layers.Conv2DTranspose(3, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh') # (bs, 256, 256, 3)
    
    x = inputs
    
    # Downsampling through the model
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)
        
    skips = reversed(skips[:-1])
    
    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = tf.keras.layers.Concatenate()([x, skip])
        
    x = last(x)
    
    return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
with strategy.scope():
    # Instantiate generators
    G_PtoM = Generator()
    G_MtoP = Generator()
    # Instantiate discriminators
    D_P = Discriminator()
    D_M = Discriminator()

In [ ]:
tf.keras.utils.plot_model(G_PtoM.layers[1], dpi=64, to_file="downsample.png", show_layer_names=False)

In [ ]:
tf.keras.utils.plot_model(G_PtoM.layers[-3], dpi=64, to_file="upsample.png", show_layer_names=False)

In [ ]:
tf.keras.utils.plot_model(G_PtoM, show_shapes=True, dpi=64, to_file="generator.png")

In [ ]:
tf.keras.utils.plot_model(D_P, show_shapes=True, dpi=64, to_file="discriminator.png")

In [ ]:
LAMBDA = 10

In [ ]:
with strategy.scope():
    loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM)

In [ ]:
def discriminator_loss(real, generated):
    real_loss = loss_object(tf.ones_like(real), real)
    generated_loss = loss_object(tf.zeros_like(generated), generated)
    total_disc_loss = real_loss + generated_loss
    total_disc_loss /= len(real)
    
    return total_disc_loss * 0.5

In [ ]:
def generator_loss(generated):
    return loss_object(tf.ones_like(generated), generated)/len(generated)

In [ ]:
def calc_cycle_loss(real_image, cycled_image): 
        return LAMBDA * tf.reduce_mean(tf.abs(real_image - cycled_image))

In [ ]:
def identity_loss(real_image, same_image):
    return LAMBDA * 0.5 * tf.reduce_mean(tf.abs(real_image - same_image))

In [ ]:
with strategy.scope():
    G_MtoP_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    G_PtoM_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

    D_M_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    D_P_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
def generate_images(model, model_r, test_input, figsize=(12,12)):
    prediction = model(test_input)
    reconstruction = model_r(prediction)
    identity = model_r(test_input)
    
    display_list = [test_input[0], prediction[0], reconstruction[0], identity[0]]
    plt.figure(figsize=figsize)
    title = ['Input', 'Predicted', 'Reconstructed', 'Identity']

    for i in range(4):
        plt.subplot(1, 4, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')

    plt.show()
    
    return display_list

In [ ]:
for images in monet_dataset.take(400):
    generate_images(G_MtoP, G_PtoM, images)

In [ ]:
with strategy.scope():
    G_MtoP_loss = tf.keras.metrics.Mean(name='G_MtoP_loss')
    G_PtoM_loss = tf.keras.metrics.Mean(name='G_PtoM_loss')
    D_M_loss = tf.keras.metrics.Mean(name='D_M_loss')
    D_P_loss = tf.keras.metrics.Mean(name='D_P_loss')

In [ ]:
def train_step(real_m, real_p):
    
    with tf.GradientTape(persistent=True) as tape:
        
        # G_PtoM translates P -> M
        # G_MtoP translates M -> M
        
        fake_m = G_PtoM(real_p, training=True)
        cycled_p = G_MtoP(fake_m, training=True)
        
        fake_p = G_MtoP(real_m, training=True)
        cycled_m = G_PtoM(fake_p, training=True)
        
        # same_m and same_p for identity loss
        same_m = G_PtoM(real_m, training=True)
        same_p = G_MtoP(real_p, training=True)
        
        # disctiminator outputs
        disc_real_m = D_M(real_m, training=True)
        disc_real_p = D_P(real_p, training=True)
        
        disc_fake_m = D_M(fake_m, training=True)
        disc_fake_p = D_P(fake_p, training=True)
        
        # Calculate Loss
        gen_MtoP_loss = generator_loss(disc_fake_p)
        gen_PtoM_loss = generator_loss(disc_fake_m)
        
        total_cycle_loss = calc_cycle_loss(real_m, cycled_m) + calc_cycle_loss(real_p, cycled_p)
        
        identity_loss_p = identity_loss(real_p, same_p)
        identity_loss_m = identity_loss(real_m, same_m)
        
        # Total Loss
        total_gen_MtoP_loss = gen_MtoP_loss + total_cycle_loss + identity_loss_p
        total_gen_PtoM_loss = gen_PtoM_loss + total_cycle_loss + identity_loss_m
        
        disc_p_loss = discriminator_loss(disc_real_p, disc_fake_p)
        disc_m_loss = discriminator_loss(disc_real_m, disc_fake_m)
        
    
    # Calculate Gradients
    gen_mtop_gradients = tape.gradient(total_gen_MtoP_loss, G_MtoP.trainable_variables)
    gen_ptom_gradients = tape.gradient(total_gen_PtoM_loss, G_PtoM.trainable_variables)
    disc_m_gradients = tape.gradient(disc_m_loss, D_M.trainable_variables)
    disc_p_gradients= tape.gradient(disc_p_loss, D_P.trainable_variables)
    
    # Apply Gradients to optimizers
    G_MtoP_optimizer.apply_gradients(zip(gen_mtop_gradients, G_MtoP.trainable_variables))
    G_PtoM_optimizer.apply_gradients(zip(gen_ptom_gradients, G_PtoM.trainable_variables))
    D_M_optimizer.apply_gradients(zip(disc_m_gradients, D_M.trainable_variables))
    D_P_optimizer.apply_gradients(zip(disc_p_gradients, D_P.trainable_variables))
    
    # Update Running Loss
    D_M_loss.update_state(disc_m_loss)
    D_P_loss.update_state(disc_p_loss)
    G_MtoP_loss.update_state(total_gen_MtoP_loss)
    G_PtoM_loss.update_state(total_gen_PtoM_loss)

In [ ]:
@tf.function
def distributed_train_step(real_m, real_p):
    strategy.run(train_step, args=(real_m, real_p))

In [ ]:
# sample_m = next(iter(monet_dataset))
# sample_p = next(iter(photo_dataset))

# ptom_preds_images = []
# mtop_reconstructions_images = []
# ptop_identity_images = []

# mtop_preds_images = []
# ptom_reconstructions_images = []
# mtom_identity_images = []

EPOCHS = 500

for epoch in range(EPOCHS):
    
#     clear_output()
#     pmp_display_list = generate_images(G_PtoM, G_MtoP, sample_p)
#     ptom_preds_images.append(pmp_display_list[1])
#     mtop_reconstructions_images.append(pmp_display_list[2])
#     ptop_identity_images.append(pmp_display_list[3])
#     
#     mpm_display_list = generate_images(G_MtoP, G_PtoM, sample_m)
#     mtop_preds_images.append(mpm_display_list[1])
#     ptom_reconstructions_images.append(mpm_display_list[2])
#     mtom_identity_images.append(mpm_display_list[3])
    
    G_MtoP_loss.reset_states()
    G_PtoM_loss.reset_states()
    D_M_loss.reset_states()
    D_P_loss.reset_states()
    
    ds = tf.data.Dataset.zip((photo_dataset, monet_dataset))
    with tqdm(ds, desc="Epoch {}/{}".format(epoch+1, EPOCHS)) as t:
        for image_p, image_m in t:
            distributed_train_step(image_m, image_p)
            t.set_description(
                f"Epoch: {epoch+1}/{EPOCHS}, "
                f"G_MtoP: {G_MtoP_loss.result():.4f}, "
                f"G_PtoM: {G_PtoM_loss.result():.4f}, "
                f"D_M: {D_M_loss.result():.4f}, "
                f"D_P: {D_P_loss.result():.4f}"
            )

In [ ]:
test_photo_dataset = tf.data.TFRecordDataset(photo_tfrec_files)
test_photo_dataset = test_photo_dataset.map(parse_tfrecord, num_parallel_calls=AUTOTUNE)
test_photo_dataset = test_photo_dataset.map(normalize, num_parallel_calls=AUTOTUNE)
test_photo_dataset = test_photo_dataset.batch(BATCH_SIZE)
# test_photo_dataset = photo_dataset

In [ ]:
plt.figure(figsize=(20,20))
for images in test_photo_dataset.take(1):
    predictions = G_PtoM(images, training=False)
    for i in range(len(images)):
        plt.subplot(8,8, 2*i+1)
        plt.imshow((images[i]+1)/2)
        plt.title("Photo")
        plt.axis('off')

        plt.subplot(8,8, 2*i+2)
        plt.imshow((predictions[i]+1)/2)
        plt.title("To Monet")
        plt.axis('off')

In [ ]:
def get_image(arr):
    arr = (arr + 1) * 127.5
    arr = tf.cast(arr, tf.int8)
    return tf.keras.preprocessing.image.array_to_img(arr)

In [ ]:
from zipfile import ZipFile

file_index = 1
with ZipFile('images.zip', 'w') as submission_zip:
    for images in tqdm(test_photo_dataset):
        predictions = G_PtoM.predict(images)
        for prediction in predictions:
            filename = f'photo_to_monet_{file_index}.jpg'
            img = get_image(prediction)
            img.save(filename)
            submission_zip.write(filename)
            os.remove(filename)
            file_index+=1
        if file_index>=9950:
            break
    submission_zip.close()
    
print(f"Saved {file_index} files in images.zip")